### Bytes literal & Encoding

- Send and Receive bytes

In [ ]:
b"hello world" == "hello world".encode()

### Socket Programming with Python

A **socket address** is an end-point for data communication. It consists of an ip address and a port number

Development of application programs that makes use of these socket address to communicate is called **socket/network programming**.

The most common type for socket applications are client-server applications, where we have a server part and a client part. As the name suggests:
- the server part wait for requests from the clients
- clients request services from the servers.

### Server Program
To code an iterative server program in Python, we carry out the following in steps:
1. import `socket` module, 
2. create a `socket` object, default is ipv4, tcp.  [see reference](https://docs.python.org/3/library/socket.html#constants)
3. assign an IP address and port number to the socket object using the `bind()` method. `bind()` takes in a tuple of IP address and port number `(ip,port)`. If `ip=''`, then the socket will listen from *any computer* in the network. Take note that port numbers 0 to 1023 should be avoided as they're reserved. In A-Level, you're only required to create a server program that communicates with interacts with a client program on the same computer, so `ip= '127.0.0.1'`, which refers to localhost. 
4. Puts the socket program into passive mode by creating a queue of n incoming connections by using the listen(n), default is 128
5. using a while loop, we accept incoming connections to that socket using `accept()` method. `accept()` returns a tuple consisting of :
    - a `connection socket` object
    - a tuple consistent of `(ip_adress, port_number)` of the remote client.
6. with the `connection socket` object from part 5, we can now either:
    - receive message using `recv(n)` method. The parameter n is an integer indicating maximum amount of data to retrieved from the receive buffer. The operation blocks when there are no data in the buffer and returns b'' when the connection is terminated by the sender
    - send message using `send()` method. `send()` takes in `bytes` object as parameters, as such we need to encode the data first using `encode()` method.
7. process the data as needed
8. close the connection by using `close()` method on the connection object.


In [ ]:
## Echo Server, just print the message from the Client

from socket import socket

listen_socket = socket() ## ipv4, TCP
listen_socket.bind( ('127.0.0.1', 1234) )
print("Server in listen mode ...")
listen_socket.listen() ## default argument
while True:
    client_socket, remote_address = listen_socket.accept() ## blocks and wait for client connection
    print( f"Acception connection from {remote_address[0]}:{remote_address[1]}")
    message = client_socket.recv(4096) ## max size from client is 4K
    message = message.decode()
    print(message)
    client_socket.send(b"OK")
    client_socket.close()

    if message == "bye":
        break
listen_socket.close()
print("Closing listen socket")

### Client Program
Coding a client program in Python is very similar to making a server program, the difference is that the client program doesn't have to be constantly listening to request. As such, we carry out the following in sequence:
1. import `socket` module, 
2. create a `socket` object,

***bind() is optional and is not needed on a client, the socket module will choose the source address and port number for you***

3. connect to the desired IP address and port number through the socket object using the `connect()` method. `connect()` takes in a tuple of IP address and port number `(ip,port)`. For A-Level purposes, we can again set `ip` to be `'127.0.0.1'`
4. send and receive data with `send()` and `receive()` method,
5. close the connection by using `close()` method on the connection object.
6. process the data as needed.


#### Exercise 1:

Write a Client and Iterative Server "Guess the number game".

Server Code:

1. Server starts and listens for incoming client connection. 
2. When the server receives a client connection, it generates a random number from 1 to 100 
3. Send a message to the client to guess a number
4. Wait for client to send number
5.      - reply "Correct" and end game if number is correct
        - reply "Low" if number is too small
        - reply "High" if number is too big
6. Repeat 3 till number is guessed correctly

Client Code:

1.  Connect to the Server
2.  Wait for the server to start game
3.  Guess the number until the server returns "Correct"

In [ ]:
### Guess Server Code
## Naren
from socket import socket
import random
ran_number = random.randint(1,100)
listen_socket = socket() ##ipv4, TCP
listen_socket.bind(('127.0.0.2', 3333 ))## or any 127 address or empty string
listen_socket.listen(1) ## default argument when empty
client_socket, remote_address = listen_socket.accept() ## blocks and wait for client connection, a (socket,(client_ip,port_number))
print(f"Accepting connection from ({remote_address[0]} : {remote_address[1]})")
while True :
    client_socket.send(b"Guess the number")
    message = client_socket.recv(4096) ## max size from client is 4kB
    message = message.decode()
    if int(message) == ran_number :
        client_socket.send(b"Correct")
        break
    elif int(message) < ran_number :
        client_socket.send(b"Low")
    else :
        client_socket.send(b"High")

print("Closing listening socket")
listen_socket.close()

In [ ]:
### Guess Client Code
## Naren
from socket import socket
s = socket()
s.connect(('127.0.0.2',3333))
while True :
    server_message = s.recv(4096).decode()
    print(server_message)
    guess = input("Input your guess : ")
    s.send(guess.encode())
    response_to_guess = s.recv(4096).decode()
    if response_to_guess == "Correct" :
      print("You won")
      break
    else :
      print(reponse_to_guess)
s.close()

In [ ]:
## Guess Server Code (Mr Leong)
import socket
import random

listen_socket = socket.socket()
listen_socket.bind(('127.0.0.1', 9999))
listen_socket.listen()
print("Guess Server listening ...")
s, addr = listen_socket.accept()
answer = random.randint(1, 100)
print(answer)
guessed = False
while not guessed : 
    s.sendall(b'GUESS\n')
    #s.send(b'GUESS')

    data = s.recv(4096)
    guess = int(data)
    print("received", guess)
    if guess < answer:
        s.sendall(b'Low\n')
        #s.send(b'Low')
    elif guess > answer:
        s.sendall(b'High\n')
        #s.send(b'High')
    else:
        guessed = True
        break
if guessed:
    s.sendall(b'Correct\n')
    #s.send(b'Correct')
s.close()
print("Server closed")

In [ ]:
## Guess Client Code (Mr Leong)
import socket, time

s = socket.socket()
s.connect(('127.0.0.1', 9999))

correct = False
while not correct :
    raw = s.recv(4096)
    print(f"Received {raw}")
    code = raw.decode()
    if code == 'Low':
        print('Your guess is too low.')
    elif code== 'High':
        print('Your guess is too high.')
    elif code == 'GUESS':
        guess = input('Enter guess (1-100): ')
        s.send(guess.encode())
    elif code == 'Correct':
        print('You guessed correct!')
        correct = True
        break
s.close()


##### Handling unknown size of message send by client

In [ ]:
## Echo Server, unknown size of received message

import socket

listen_socket = socket.socket() ## ipv4, TCP
listen_socket.bind( ('127.0.0.1', 1234) )
print("Server in listen mode ...")
listen_socket.listen() ## default argument
while True:
    client_socket, remote_address = listen_socket.accept() ## blocks and wait for client connection
    print( f"Acception connection from {remote_address[0]}:{remote_address[1]}")
    #message = client_socket.recv(4096) ## how big ?? 2, 4, 8,..1024,
    #message = message.decode()

    message =b""
    while True:
        chunk = client_socket.recv(4096)
        message += chunk
        #print(chunk.decode(), end="")
        if chr(0).encode() in chunk:
            break
    message = message.decode()
    print(message)

    ## Terminate Server
    if message[:-1] == "bye":
        break
listen_socket.close()
print("Closing listen socket")

#### Exercise 2:

Write a client and iterative server program for a client to upload a file to the server.

The server will 

- listen for incoming client connection
- upon receving a client connection the server will receive the contents of the file and the filename to be saved from the client and then save the file on the server

The client will 

- read a file and send its contents and file name to be saved to the server


In [ ]:
## Upload Server
## Jayden

from socket import socket

try:
    listen_socket = socket()
    listen_socket.bind(('127.0.0.1', 1234))
    print("Server in listen mode...")
    listen_socket.listen()

    while True:
        client_socket, remote_address = listen_socket.accept()
        print(f"Connection from {remote_address[0]}:{remote_address[1]}")
        
        msg = ""
        while True:
            chunk = client_socket.recv(4096)
            msg += chunk.decode()
            if msg[-2:] == chr(0)*2: # 2 null characters --> EOF
                print("End of file")
                break

        fn, contents = msg[:-2].split(chr(0))
        f = open(f"recv-{fn}","w")
        f.write(contents)
        f.close()

        client_socket.sendall((chr(0).encode())*2)

except KeyboardInterrupt: # Exits
    listen_socket.close()
    print("Closed listen socket")

In [ ]:
## Upload Client
## Jayden

from socket import socket
fn = input("Enter file: ")

f = open("fn")
s = socket()
s.connect(('127.0.0.1',1234))

contents = f.read()
f.close()

s.sendall(fn.encode())
s.sendall(chr(0).encode()) # Delimiter
s.sendall(contents.encode())
s.sendall(chr(0).encode()*2) # EOF
print("Sent file")

while True:
    rec_msg = s.recv(1024)
    if rec_msg == chr(0).encode()*2:
        s.close()
        break
print("Received acknowledgement")

____
##### Handling concurrent clients

In [ ]:
## Multi-threaded Echo Server

import socket
import threading

class echoThread(threading.Thread):
    def __init__(self,client_socket, remote_address ):
        super().__init__()
        self.client_socket = client_socket
        self.remote_address = remote_address

    def run(self):
        print( f"Acception connection from {self.remote_address[0]}:{self.remote_address[1]}")
        message =b""
        while True:
            chunk = self.client_socket.recv(4096)
            message += chunk
            print(chunk.decode(), end="")
            if chr(0).encode() in chunk:
                break
        message = message.decode()
        self.client_socket.close()

listen_socket = socket.socket() ## ipv4, TCP
listen_socket.bind( ('127.0.0.1', 1234) )
print("Server in listen mode ...")
listen_socket.listen() ## default argument
while True:
    client_socket, remote_address = listen_socket.accept() ## blocks and wait for client connection
    client = echoThread(client_socket, remote_address)    
    client.start()



### HBL 3 Exercise 24.6 2020/NJC/P2/Q1 H2 Computing
[Link](Networking_Exercises.ipynb)